In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/sa_update"
%pwd
import os
print(os.listdir())

Mounted at /content/drive
/content/drive/MyDrive/sa_update
['.ipynb_checkpoints', '__pycache__', '전세가율_수정.csv', 'sa_update.ipynb', 'kqc_custom.py', '월별동별평균가격(113개동).csv', '월별동별평균가격변화율(113개동).csv', '전체데이터(113개동).csv', '무위험만기이자수익률(국고채 3년).csv', '서울시전월대비가격변화율.csv', '제발.csv', '제발2.csv', '전세가율_최종.csv', 'logdata.csv', 'SA_basefunctions.py', 'SA_ver2.py']


In [2]:
!pip install qiskit
!pip install qiskit_optimization
!pip install qiskit.utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 5.0 MB/s 
     |████████████████████████████████| 19.2 MB 12.4 MB/s 
     |████████████████████████████████| 240 kB 61.6 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 106 kB 51.1 MB/s 
     |████████████████████████████████| 929 kB 78.9 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 37.5 MB 512 kB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 4.0 MB 55.3 MB/s 
     |████████████████████████████████| 1.9 MB 44.9 MB/s 
     |████████████████████████████████| 112 kB 66.1 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.3-py3-none-any.whl size=12262 sha256=70a28fb0b6ecfb998e8673f0128bd997df15dca4cdc5f5330b5a202e96443bef
  Stored in directory: /root/.cache/pip/wheels/46/24/40/7b6f

In [3]:
from kqc_custom import generate_dependent_sample
from SA_ver2 import SimulatedAnnealing as SA
import matplotlib.pyplot as plt
from SA_basefunctions import get_QB as qb
import numpy as np
import pandas as pd

In [4]:
# 데이터 불러오기
data =  pd.read_csv(('전체데이터(113개동).csv'),index_col=0)
data_price = pd.read_csv(('월별동별평균가격(113개동).csv'),index_col=0)
data_price_change = pd.read_csv(('월별동별평균가격변화율(113개동).csv'),index_col=0)
# 무위험 이자율 + 
data_seoul = pd.read_csv(('서울시전월대비가격변화율.csv'),index_col=0)
data_risk = pd.read_csv(('무위험만기이자수익률(국고채 3년).csv'),index_col=0)
# 119(기간) x 113(동) 
data_price_change
data = data_price_change
# 119 x 1
#data_seoul

In [5]:
# Test vs Training :: 7 - 3
training_data = data_price_change[:84]
test_data = data_price_change[84:]

In [6]:
X  = training_data
X.columns = range(0,113)
y=data_seoul[:84]

In [7]:
# 회귀계수로 베타값들 구하기
# 전체 베타

beta = []
for i in range(113) :
    beta.append( np.polyfit( data_seoul["거래금액(만원)"]-data_risk["국고채(3년,평균)"],data[data.columns[i]]-data_risk["국고채(3년,평균)"].values  ,1 )[0] )
    

# training 베타
beta_training = []
for i in range(113) :
    beta_training.append( np.polyfit( data_seoul["거래금액(만원)"][:84]-data_risk["국고채(3년,평균)"][:84],data[data.columns[i]][:84]-data_risk["국고채(3년,평균)"][:84].values  ,1 )[0] )

# test 베타
beta_test = []
for i in range(113) :
    beta_test.append( np.polyfit( data_seoul["거래금액(만원)"][84:]-data_risk["국고채(3년,평균)"][84:],data[data.columns[i]][84:]-data_risk["국고채(3년,평균)"][84:].values  ,1 )[0] )   

In [8]:
beta_training = np.array(beta_training)
Q=training_data.corr()

lst = [100, 100, 100, 200, 200, 200, 200, 300, 300, 300, 300, 300, 400, 400, 400, 400, 400, 400]

k=2
alpha=0.9
tau=1
ld = 0.5

In [9]:
X

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
1,-0.080720,-0.156774,0.011721,-0.063545,0.082297,0.394790,-0.043526,0.051611,0.032827,0.198089,...,0.013177,0.652592,0.003870,-0.250777,0.033961,0.063326,-0.046875,0.094138,0.253235,-0.090766
2,0.176547,0.126996,0.124024,0.008929,0.005154,-0.122238,-0.343329,-0.026184,-0.078333,-0.108090,...,0.195568,-0.231124,0.061742,0.207464,0.112809,0.091902,0.016393,0.030504,0.199412,-0.051960
3,-0.087963,0.101102,-0.138423,0.182965,-0.003032,0.195424,0.400487,0.121840,0.061980,0.003362,...,0.058816,-0.000089,0.013383,-0.009527,-0.114976,-0.069518,0.012903,0.197942,0.000000,0.012456
4,-0.015050,0.055868,-0.089162,-0.120884,-0.024204,-0.137913,0.181865,-0.120002,0.175581,0.084887,...,-0.079212,0.135942,-0.059675,-0.108275,0.006463,0.062332,-0.022824,-0.071803,-0.386923,0.005249
5,-0.016784,-0.155543,0.215233,0.313110,-0.048761,0.019793,-0.034712,0.089072,-0.132248,-0.091445,...,0.056200,-0.149454,0.006242,-0.030765,0.078421,-0.037173,-0.008419,0.031852,0.273526,0.025198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,-0.230061,-0.145650,-0.058065,-0.067528,0.486032,-0.079340,0.090725,0.008201,-0.259454,-0.114191,...,-0.026351,0.017668,0.008611,0.274742,-0.024499,0.162756,0.064674,0.082548,-0.013699,0.159085
81,0.128287,0.041762,0.020369,-0.001604,0.319013,0.557364,0.015800,-0.049245,0.347698,0.104021,...,-0.100788,-0.006027,-0.000179,0.084497,0.068910,-0.329247,-0.119448,-0.186908,-0.138889,-0.071954
82,-0.045808,0.047776,-0.091299,-0.009746,0.039800,-0.131855,-0.007329,0.079929,-0.125318,0.000970,...,-0.148940,-0.099729,0.105517,0.242917,0.019489,0.179576,0.274444,0.151813,0.580645,0.072920
83,0.036937,0.259119,0.111763,-0.016189,-0.217436,0.060417,-0.123863,0.131385,0.068000,-0.182961,...,0.394498,0.099657,0.022833,-0.231932,-0.001775,0.015670,-0.113339,-0.011892,-0.241537,-0.060652


In [10]:
for i in range (10):
    simulatedannealing = SA(X, y, lst, k, alpha, 0.95, Q = Q, Beta = beta_training, Lambda = ld, constraint = 10, condition = [0,1,2,3,4,5,6,7] )
    sa_result= simulatedannealing.equal()
    print(sa_result.columns)
    print(qb(sa_result, 113, Q, beta_training, ld))

/content/drive/MyDrive/sa_update/SA_ver2.py:45: RuntimeWarning: overflow encountered in exp
  if np.random.rand(1) <= min(1, np.exp((obj(self.x.iloc[:, np.where(theta_zero)[0]], p, Q = self.Q, beta = self.Beta, lmbd = self.Lambda)-obj(self.x.iloc[:, np.where(theta_star)[0]], p, Q = self.Q, beta = self.Beta, lmbd = self.Lambda))/tau)):


Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 26, 70], dtype='int64')
2.692887302978313


In [32]:
for i in range (10):
    simulatedannealing = SA(X, y, lst, k, alpha, tau, Q = Q, Beta = beta_training, Lambda = ld, constraint = 7, condition = [26, 70] )
    sa_result = simulatedannealing.equal()
    print(sa_result.columns)
    print(qb(sa_result, 113, Q, beta_training, ld))

Int64Index([26, 30, 38, 70, 79, 87, 91], dtype='int64')
-0.7259452827461566
Int64Index([15, 26, 38, 67, 70, 80, 87], dtype='int64')
-0.9721816679509998
Int64Index([26, 32, 38, 46, 70, 72, 87], dtype='int64')
-1.0200148818033237
Int64Index([26, 32, 38, 46, 70, 72, 87], dtype='int64')
-1.0200148818033237
Int64Index([26, 45, 49, 70, 72, 79, 87], dtype='int64')
-0.8082974101914868
Int64Index([26, 32, 38, 46, 70, 72, 87], dtype='int64')
-1.0200148818033237
Int64Index([16, 26, 38, 46, 70, 80, 87], dtype='int64')
-1.0691804710186226
Int64Index([16, 26, 38, 46, 70, 80, 87], dtype='int64')
-1.0691804710186226
Int64Index([26, 32, 38, 46, 70, 72, 87], dtype='int64')
-1.0200148818033237
Int64Index([16, 26, 38, 46, 70, 80, 87], dtype='int64')
-1.0691804710186226


In [30]:
for i in range (10):
    simulatedannealing = SA(X, y, lst, k, alpha, 0.95, Q = Q, Beta = beta_training, Lambda = ld, constraint = 113)
    sa_result= simulatedannealing.unequal()
    print(sa_result.columns)
    print(qb(sa_result, 113, Q, beta_training, ld))

Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 72, 80, 85, 87, 98], dtype='int64')
-1.2869600178918699
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
Int64Index([26, 46, 49, 72, 80, 87], dtype='int64')
-1.2753962737145839
Int64Index([16, 26, 46, 72, 80, 87, 101], dtype='int64')
-1.2857239542992827
Int64Index([26, 49, 72, 80, 87, 98], dtype='int64')
-1.343436233949303
